# Assignment 1

## Data Preprocessing

### Imports

In [ ]:
import os
import requests
import zipfile
from tqdm import tqdm
import time
import datetime
from IPython.display import display

from typing import List, Dict, Tuple

import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, \
                            f1_score, recall_score, ConfusionMatrixDisplay

import nltk
from nltk.corpus import stopwords
import gensim
import gensim.downloader as gloader
assert int(gensim.__version__.split('.')[0]) >= 4, "Install gensim 4.x.x or above (pip install -U gensim)"

nltk.download('stopwords')
%matplotlib inline

L2_REG = 0.0001
DROPOUT_RATE = 0.2
BATCH_SIZE = 128

### Downloading the Dataset

In [2]:
ROOT_PATH = os.path.dirname(os.getcwd())
DATA_PATH = os.path.join(ROOT_PATH, 'data')

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

def download_data(data_path):
    final_path = os.path.join(data_path, 'dependency_treebank.zip')
    url = "https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip"

    if not os.path.exists(data_path):
        os.makedirs(data_path)

    if not os.path.exists(final_path):
        print("Downloading dataset...")
        with requests.Session() as current_session:
            response = current_session.get(url, stream=True)
        save_response_content(response, final_path)
        print("Download completed!")

        print("Extracting dataset...")
        with zipfile.ZipFile(final_path) as loaded_zip:
            loaded_zip.extractall(data_path)
        print("Extraction completed!")

download_data(DATA_PATH)

Download completed!
Extracting dataset...
Extraction completed!


## Embeddings

## Baseline model

## Experiments

## Evaluation

## Error Analysis